In [3]:
import re
import datetime

def extract_and_format_date(text):
    """Extracts and formats dates from various text formats.

    Args:
        text: The input text containing the date.

    Returns:
        The extracted date in dd/mm/yyyy format, or None if no date is found.
    """

    # Month mapping dictionary
    month_mapping = {
        'January': '01', 'February': '02', 'March': '03', 'April': '04',
        'May': '05', 'June': '06', 'July': '07', 'August': '08',
        'September': '09', 'October': '10', 'November': '11', 'December': '12',
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
        'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
        'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }

    # Regular expressions for common date formats
    date_patterns = [
        r'\b(\d{1,2}/\d{1,2}/\d{4})\b',  # dd/mm/yyyy
        r'\b(\d{1,2}/\d{1,2}/\d{2})\b',  # dd/mm/yy
        r'\b(\d{4}/\d{1,2}/\d{1,2})\b',  # yyyy/mm/dd
        r'\b(\d{4}-\d{2}-\d{2})\b',  # yyyy-mm-dd
        r'\b(\d{1,2}-\d{1,2}-\d{4})\b',  # dd-mm-yyyy
        r'\b(\d{1,2}-\d{1,2}-\d{2})\b',  # dd-mm-yy
        r'\b(\d{1,2}\.\d{1,2}\.\d{4})\b', # dd.mm.yyyy
        r'\b(\d{1,2}\.\d{1,2}\.\d{2})\b', # dd.mm.yy
        r'\b(\d{4}\.\d{1,2}\.\d{1,2})\b', # yyyy.mm.dd
        r'(\d{1,2})(?:st|nd|rd|th)? (\w+) (\d{4})',  # e.g., 5th March 2023
        r'(\d{1,2})(?:st|nd|rd|th)? (\w+), (\d{4})',  # e.g., 5th March, 2023
        r'(\d{1,2})(?:st|nd|rd|th)? of (\w+) (\d{4})',  # e.g., 5th of March 2023
        r'(\d{1,2})(?:st|nd|rd|th)? of (\w+), (\d{4})',  # e.g., 5th of March 2023
        r'(\w+) (\d{1,2}), (\d{4})',  # e.g., March 5, 2023
        r'(\w+) (\d{1,2})(?:st|nd|rd|th)?, (\d{4})',  # e.g., March 5th, 2023
        r'(\d{1,2})(?:st|nd|rd|th)? (\w{3}) (\d{4})',  # e.g., 5th Mar 2023
    ]

    for pattern in date_patterns:
        match = re.search(pattern, text)
        if match:
            try:
                # Handle matched date strings based on detected pattern
                date_str = match.group(0)
                # Handle different patterns
                if '/' in date_str:  # dd/mm/yyyy, dd/mm/yy, or yyyy/mm/dd
                    day, month, year = date_str.split('/')
                    if int(day) < 32 and int(month) < 13: 
                        
                        if int(year) > 1500:  # dd/mm/yyyy
                            date_obj = datetime.datetime.strptime(date_str, '%d/%m/%Y')
                        else:  # dd/mm/yy
                                date_obj = datetime.datetime.strptime(date_str, '%d/%m/%y')
                    elif int(day) < 13 and int(month) > 12 and int(year) > 1500:
                            month, day, year = date_str.split('/')
                            date_obj = datetime.datetime.strptime(date_str, '%m/%d/%Y')
                    else: 
                        year, month, day = date_str.split('/') # yyyy/mm/dd
                        date_obj = datetime.datetime.strptime(date_str, '%Y/%m/%d')
                
                elif '-' in date_str:  # dd-mm-yyyy, dd-mm-yy, or yyyy-mm-dd
                    day, month, year = date_str.split('-')
                    if int(day) < 32 and int(month) < 13: 
                        if int(year) > 1500:  # dd-mm-yyyy
                            date_obj = datetime.datetime.strptime(date_str, '%d-%m-%Y')
                        else:  # dd-mm-yy
                                date_obj = datetime.datetime.strptime(date_str, '%d-%m-%y')
                    elif int(day) < 13 and int(month) > 12 and int(year) > 1500:
                            month, day, year = date_str.split('-')
                            date_obj = datetime.datetime.strptime(date_str, '%m-%d-%Y')
                    else: 
                        year, month, day = date_str.split('-') # yyyy-mm-dd
                        date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
                
                elif '.' in date_str:  # dd.mm.yyyy, dd.mm.yy, or yyyy.mm.dd
                    day, month, year = date_str.split('.')
                    if int(day) < 32 and int(month) < 13: 
                        if int(year) > 1500:  # dd.mm.yyyy
                            date_obj = datetime.datetime.strptime(date_str, '%d.%m.%Y')
                        else:  # dd.mm.yy
                                date_obj = datetime.datetime.strptime(date_str, '%d.%m.%y')
                    elif int(day) < 13 and int(month) > 12 and int(year) > 1500:
                            month, day, year = date_str.split('.')
                            date_obj = datetime.datetime.strptime(date_str, '%m.%d.%Y')
                    else: 
                        year, month, day = date_str.split('.') # yyyy.mm.dd
                        date_obj = datetime.datetime.strptime(date_str, '%Y.%m.%d')
                
                else:  # Textual format
                    day, month_str, year = match.groups()
                    #print(day, month_str, year)
                    if day in month_mapping:  # Get month number from dictionary
                        day,month_str=month_str, day
                    #print(day, month_str, year)
                    month_str = month_str.capitalize()  # Ensure month names are properly capitalized
                    #print(month_str)
                    month = month_mapping.get(month_str)
                    day = re.sub(r'(st|nd|rd|th)', '', day).strip()  # Remove ordinal suffixes
                    #print(day, month, year)
                    
                    # Check if day and month are valid
                    if int(day) < 32 and month:
                        date_str = f"{day.zfill(2)}/{month}/{year}"
                        date_obj = datetime.datetime.strptime(date_str, '%d/%m/%Y')

                if 'date_obj' in locals():  # Ensure date_obj is defined
                    return date_obj.strftime('%d/%m/%Y')
            except ValueError:
                continue  # Handle invalid date formats

    return None

In [9]:
import pandas as pd

df=pd.read_csv('date_parser_testcases.csv')

df['date']=df['Input'].apply(extract_and_format_date)

df.to_csv('date_parser_testcases1.csv',index=False)

In [12]:
df.date.isna().sum() #originally 100 rows

2